In [4]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

username = "aacuser"
password = "SNHU1234"
shelter = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
if '_id' in df.columns:
    df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# logo (high‑res file placed alongside this script)
try:
    encoded_image = base64.b64encode(open('grazioso_logo.png', 'rb').read()).decode()
    logo = html.Img(src=f'data:image/png;base64,{encoded_image}', style={'height':'48px'})
except Exception:
    logo = html.Div('Grazioso Salvare')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),

    html.Header(
        [
            logo,
            html.Div("AJ", style={
                "width":"40px","height":"40px","borderRadius":"8px",
                "display":"flex","alignItems":"center","justifyContent":"center",
                "background":"linear-gradient(135deg, purple, cyan)","color":"white",
                "fontWeight":"bold","fontSize":"16px","marginRight":"10px"
            }),
            html.H2("SNHU CS-340 Dashboard", style={"margin":"0","fontSize":"18px"})
        ],
        id="dashboard-identifier",
        style={"display":"flex","alignItems":"center","gap":"8px","margin":"8px 0"}
    ),

    html.Hr(),

    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label':'Water Rescue','value':'water'},
            {'label':'Mountain/Wilderness Rescue','value':'mountain'},
            {'label':'Disaster/Individual Tracking','value':'disaster'},
            {'label':'Reset (All)','value':'reset'}
        ],
        value='reset',
        inline=True
    ),

    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        
        page_size=10,
        sort_action="native",
        filter_action="native",
        row_selectable="single",
        selected_rows=[0] if len(df) else [],
        style_table={'overflowX': 'auto'}

    ),
    html.Br(),
    html.Hr(),
    html.Div([
        html.Div(id='graph-id', className='col s12 m6', style={'flex':1, 'paddingRight':'10px'}),
        html.Div(id='map-id', className='col s12 m6', style={'flex':1, 'paddingLeft':'10px'})
    ], style={'display':'flex','flexWrap':'wrap'})
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# update table from filter selection
@app.callback(Output('datatable-id','data'), [Input('filter-type','value')])
def update_table(filter_type):
    if filter_type == 'water':
        query = {
            "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},
            "sex_upon_outcome":"Intact Female",
            "age_upon_outcome_in_weeks": {"$gte":26, "$lte":156}
        }
    elif filter_type == 'mountain':
        query = {"breed": {"$in": ["German Shepherd","Alaskan Malamute","Siberian Husky","Rottweiler"]}}
    elif filter_type == 'disaster':
        query = {"breed": {"$in": ["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]}}
    else:
        query = {}
    dff = pd.DataFrame.from_records(shelter.read(query))
    if '_id' in dff.columns:
        dff.drop(columns=['_id'], inplace=True)
    return dff.to_dict('records')

# chart updates from table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    dff = pd.DataFrame(viewData) if viewData else pd.DataFrame()
    if dff.empty or 'breed' not in dff.columns:
        return [html.Div("No data available")]
    counts = dff['breed'].dropna().astype(str).value_counts().head(10).reset_index()
    counts.columns = ['breed','count']
    fig = px.bar(counts, x='breed', y='count', title='Top Breeds (Top 10)')
    return [dcc.Graph(figure=fig)]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if not viewData:
        return [html.Div("No data yet…")]
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if (index and len(index) > 0) else 0
    row = max(0, min(row, len(dff) - 1))
    try:
        lat = float(dff.iloc[row].get('location_lat', 30.75))
        lon = float(dff.iloc[row].get('location_long', -97.48))
    except (TypeError, ValueError):
        lat, lon = 30.75, -97.48
    name = dff.iloc[row].get('name', '(unknown)')
    if pd.isna(name) or (isinstance(name, str) and name.strip() == ''):
        name = '(unknown)'
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(name),
                dl.Popup([
                    html.H3("Animal Name"),
                    html.P(name)
                ])
            ])
        ])
    ]

app.run_server(debug=True)


Dash app running on http://127.0.0.1:11000/
